In [5]:
import torch
import sys
from torch.utils.data import DataLoader
sys.path.append('data/')
sys.path.append('models/')
from my_dataset import MyDataset00000000 as MyDataset
from models import BaseModel
import torch.nn.functional as F
import torch.nn as nn


In [6]:


class MyModel(BaseModel):

    def __init__(self, input_size=128, hidden_size=512, num_layers=1, dropout_rate=0.2, **kwargs):
        super().__init__()
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        # Define the lstm layer
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        
        # Define the batch normalization layer
        self.batchnorm = nn.BatchNorm1d(hidden_size)
        
        # Define the dropout layer
        self.dropout = nn.Dropout(dropout_rate)

        # Define the ReLU activation function
        self.relu = nn.ReLU()

        self.task_layers = nn.ModuleDict({
            "short_mortality": nn.Linear(hidden_size, 2),
            "long_mortality": nn.Linear(hidden_size, 2),
            "readmission": nn.Linear(hidden_size, 2),
            "diagnosis": nn.Linear(hidden_size, 34), # 17 diagnoses, each binary
            "short_los": nn.Linear(hidden_size, 2),
            "long_los": nn.Linear(hidden_size, 2),
            "final_acuity": nn.Linear(hidden_size, 6),
            "imminent_discharge": nn.Linear(hidden_size, 6),
            "creatinine_level": nn.Linear(hidden_size, 5),
            "bilirubin_level": nn.Linear(hidden_size, 5),
            "platelet_level": nn.Linear(hidden_size, 5),
            "wbc_level": nn.Linear(hidden_size, 3),
        })
        
    def get_logits(cls, net_output):
        logits = []
        idx = 0
        for task_size in cls.task_sizes:
            logits.append(net_output[:, idx:idx + task_size])
            idx += task_size
        return logits
    
    def get_targets(self, sample):
        return sample["labels"]
    
    def forward(self, x, **kwargs):

        x = x.view(-1, 100, 128)

        # Initialize hidden and cell state
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        
        # Pass the input through the LSTM layer
        x, _ = self.lstm(x, (h0, c0))
        
        # Apply batch normalization
        x = self.batchnorm(x[:, -1, :])

        # Apply ReLU activation function
        x = self.relu(x)
        
        # Apply dropout
        x = self.dropout(x)
        
        # Pass the output through the task specific layers
        output = {}
        for task, layer in self.task_layers.items():
            output[task] = layer(x)

        return output
        
    def multitask_loss(self, cls, predictions, targets):
        # Define loss functions for each task
        loss_functions = {
            "short_mortality": nn.CrossEntropyLoss(),
            "long_mortality": nn.CrossEntropyLoss(),
            "readmission": nn.CrossEntropyLoss(),
            "diagnosis": nn.BCEWithLogitsLoss(),
            "short_los": nn.CrossEntropyLoss(),
            "long_los": nn.CrossEntropyLoss(),
            "final_acuity": nn.CrossEntropyLoss(),
            "imminent_discharge": nn.CrossEntropyLoss(),
            "creatinine_level": nn.CrossEntropyLoss(),
            "bilirubin_level": nn.CrossEntropyLoss(),
            "platelet_level": nn.CrossEntropyLoss(),
            "wbc_level": nn.CrossEntropyLoss(),
        }
        
        # Calculate loss for each task
        losses = {}
        idx = 0
        for task, prediction in predictions.items():
            i = cls[task]
            target_task = [t[idx:i] for t in targets]
            # Reshape the target
            target_task = torch.stack(target_task)
            losses[task] = loss_functions[task](prediction, target_task)
            idx = i
        
        # Combine the losses
        total_loss = sum(losses.values())
        
        return total_loss

In [7]:
# Parameters
data_path = '/Users/noelkj/Documents/GitHub/AI612-project2/train/testdata/'
batch_size = 3
num_epochs = 1
learning_rate = 1e-3

# Create the dataset and a data loader
dataset = MyDataset(data_path=data_path)
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Instantiate the model
model = MyModel()
model.to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))

# Create an optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

cls = {
    "short_mortality": 2,
    "long_mortality": 2,
    "readmission": 2,
    "diagnosis": 34,
    "short_los": 2,
    "long_los": 2,
    "final_acuity": 6,
    "imminent_discharge": 6,
    "creatinine_level": 5,
    "bilirubin_level": 5,
    "platelet_level": 5,
    "wbc_level": 3,
}

Loading data from /Users/noelkj/Documents/GitHub/AI612-project2/train/testdata/ to cpu


In [8]:

# Training loop
for epoch in range(num_epochs):
    for batch in data_loader:
        # Unpack the data and move it to the appropriate device
        inputs = batch["data"]
        targets = batch["label"]

        # Forward pass
        outputs = model(inputs)

        # Compute the loss
        loss = model.multitask_loss(cls, outputs, targets)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Print the loss for the current epoch
    print(f"Epoch {epoch + 1}/{num_epochs} - Loss: {loss.item()}")

RuntimeError: Expected floating point type for target with class probabilities, got Long